# Naive Bayes Classifier

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("train.csv")

In [3]:
df=df.sort_values(by=['is_claim'])

In [4]:
df

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
38405,ID38406,0.084490,0.03,0.346154,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
38406,ID38407,0.723504,0.04,0.394231,C8,8794,4,B2,M5,Diesel,...,No,Yes,Yes,Yes,No,No,Yes,Yes,5,0
38407,ID38408,0.898402,0.00,0.307692,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
38408,ID38409,0.370271,0.05,0.326923,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34861,ID34862,0.512484,0.10,0.326923,C17,65567,4,B2,M5,Diesel,...,No,Yes,Yes,Yes,No,No,Yes,Yes,5,1
51164,ID51165,0.414008,0.06,0.336538,C8,8794,1,C1,M2,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,1
34866,ID34867,0.870057,0.04,0.596154,C2,27003,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,1
6955,ID06956,0.505479,0.09,0.432692,C10,73430,1,B2,M6,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,1


In [5]:
round(len(df[df['is_claim']==0])*0.2)

10969

In [6]:
round(len(df[df['is_claim']==1])*0.2)

750

In [7]:
import random
def shuffle(df):
    temp=list(range(len(df)))
    random.shuffle(temp)
    df=df.iloc[temp,:]
    return(df)

In [8]:
random.seed(6111058)
df_0=df[df['is_claim']==0]
df_1=df[df['is_claim']==1]
df_0=shuffle(df_0)
df_1=shuffle(df_1)

In [9]:
test_df=pd.concat([df_0[:10969],df_1[:750]])

In [10]:
train_df=pd.concat([df_0[10969:],df_1[750:]])

In [11]:
train_feature=train_df.drop(['policy_id','is_claim'],axis=1)
train_label=train_df['is_claim']

In [12]:
import math
def naive_bayes(df,label,test_df):
    df=pd.concat([df,label],axis=1)
    beta=[]
    for i in range(len(df.dtypes)-1):
        if df.dtypes[i] == float or df.dtypes[i] == np.int64:
            temp_0=df[df.iloc[:,42]==0]
            temp_1=df[df.iloc[:,42]==1]
            mean0=temp_0.iloc[:,i].mean()
            std0=temp_0.iloc[:,i].std()
            mean1=temp_1.iloc[:,i].mean()
            std1=temp_1.iloc[:,i].std()
            beta.append([[mean0,std0],[mean1,std1]])
        else:
            beta.append([[pd.DataFrame(temp_0.iloc[:,i].value_counts()/len(temp_0.iloc[:,i])).values.tolist(),list(pd.DataFrame(temp_0.iloc[:,i].value_counts()/len(temp_0.iloc[:,i])).index)],[pd.DataFrame(temp_1.iloc[:,i].value_counts()/len(temp_1.iloc[:,i])).values.tolist(),list(pd.DataFrame(temp_1.iloc[:,i].value_counts()/len(temp_1.iloc[:,i])).index)]])
    predvalue=[]
    for i in range(len(test_df)):
        score0=len(temp_0)/(len(temp_0)+len(temp_1))
        score1=len(temp_1)/(len(temp_0)+len(temp_1))
        for j in range(len(test_df.dtypes)):
            if test_df.dtypes[j] == float or test_df.dtypes[j] == np.int64:
                score0=score0*((1/math.sqrt(2*math.pi*(beta[j][0][1]**2)))*(math.exp(((test_df.iloc[i][j]-beta[j][0][0])**2)/(2*(beta[j][0][1]**2))*(-1))))
                score1=score1*((1/math.sqrt(2*math.pi*(beta[j][1][1]**2)))*(math.exp(((test_df.iloc[i][j]-beta[j][1][0])**2)/(2*(beta[j][1][1]**2))*(-1))))
            else:
                for k in range(len(beta[j][0][1])):
                    if test_df.iloc[i][j]==beta[j][0][1][k]:
                        score0=score0*beta[j][0][0][k][0]
                    if test_df.iloc[i][j]==beta[j][1][1][k]:
                        score1=score1*beta[j][1][0][k][0]
        if score0>=score1:
            predvalue.append(0)
        else:
            predvalue.append(1)
    return predvalue

In [13]:
test_feature=test_df.drop(['policy_id','is_claim'],axis=1)
test_label=test_df['is_claim']

In [14]:
bayespred=naive_bayes(train_feature,train_label,test_feature)

In [15]:
from sklearn.metrics import confusion_matrix
confusion_matrix(bayespred,test_label)

array([[10969,   750],
       [    0,     0]], dtype=int64)

# Random Forest

In [16]:
change_df=[]
for i in range(len(list(df.drop(['policy_id','is_claim'],axis=1)))):
    temp=[]
    drop_df=df.drop(['policy_id','is_claim'],axis=1)
    if drop_df.dtypes[i] == float or drop_df.dtypes[i] == np.int64:
        for j in range(len(drop_df)):
            quan=drop_df.iloc[:,i].quantile([0,0.25,0.5,0.75,1])
            if drop_df.iloc[j,i]>= quan[0] and drop_df.iloc[j,i]<quan[0.25]:
                temp.append(1)
            elif drop_df.iloc[j,i]>= quan[0.25] and drop_df.iloc[j,i]<quan[0.5]:
                temp.append(2)
            elif drop_df.iloc[j,i]>= quan[0.5] and drop_df.iloc[j,i]<quan[0.75]:
                temp.append(3)
            elif drop_df.iloc[j,i]>= quan[0.75] and drop_df.iloc[j,i]<=quan[1]:
                temp.append(4)
        change_df.append(temp)
change_df

[[2,
  1,
  3,
  3,
  2,
  2,
  3,
  2,
  2,
  2,
  2,
  2,
  2,
  3,
  1,
  1,
  1,
  2,
  3,
  2,
  4,
  4,
  4,
  3,
  2,
  4,
  4,
  1,
  2,
  2,
  2,
  1,
  1,
  4,
  1,
  1,
  4,
  2,
  1,
  4,
  2,
  1,
  2,
  4,
  1,
  4,
  3,
  1,
  1,
  3,
  3,
  4,
  1,
  2,
  1,
  2,
  3,
  1,
  4,
  4,
  1,
  3,
  1,
  2,
  3,
  4,
  4,
  3,
  4,
  3,
  3,
  4,
  4,
  1,
  2,
  1,
  3,
  1,
  1,
  1,
  2,
  2,
  4,
  3,
  2,
  2,
  4,
  4,
  4,
  3,
  4,
  3,
  4,
  1,
  2,
  3,
  4,
  4,
  2,
  4,
  1,
  3,
  1,
  2,
  3,
  4,
  3,
  2,
  1,
  2,
  1,
  2,
  2,
  2,
  1,
  3,
  2,
  4,
  1,
  3,
  1,
  1,
  1,
  3,
  3,
  3,
  1,
  4,
  4,
  1,
  3,
  2,
  2,
  1,
  4,
  4,
  2,
  2,
  3,
  1,
  1,
  1,
  2,
  1,
  4,
  3,
  4,
  1,
  2,
  3,
  1,
  1,
  2,
  1,
  2,
  4,
  2,
  4,
  2,
  3,
  2,
  1,
  1,
  2,
  2,
  3,
  2,
  2,
  2,
  1,
  1,
  3,
  4,
  2,
  2,
  2,
  4,
  2,
  2,
  2,
  2,
  3,
  1,
  3,
  1,
  1,
  2,
  1,
  3,
  3,
  2,
  4,
  4,
  4,
  1,
  4,
  2,
  4,
  3,
  3,


In [17]:
j=0
for i in range(len(list(df.drop(['policy_id','is_claim'],axis=1)))):
    temp=[]
    drop_df=df.drop(['policy_id','is_claim'],axis=1)
    if drop_df.dtypes[i] == float or drop_df.dtypes[i] == np.int64:
        print(drop_df.iloc[:,i])

0        0.515874
38405    0.084490
38406    0.723504
38407    0.898402
38408    0.370271
           ...   
34861    0.512484
51164    0.414008
34866    0.870057
6955     0.505479
28332    0.403215
Name: policy_tenure, Length: 58592, dtype: float64
0        0.05
38405    0.03
38406    0.04
38407    0.00
38408    0.05
         ... 
34861    0.10
51164    0.06
34866    0.04
6955     0.09
28332    0.11
Name: age_of_car, Length: 58592, dtype: float64
0        0.644231
38405    0.346154
38406    0.394231
38407    0.307692
38408    0.326923
           ...   
34861    0.326923
51164    0.336538
34866    0.596154
6955     0.432692
28332    0.480769
Name: age_of_policyholder, Length: 58592, dtype: float64
0         4990
38405     4076
38406     8794
38407     4076
38408     4076
         ...  
34861    65567
51164     8794
34866    27003
6955     73430
28332    34738
Name: population_density, Length: 58592, dtype: int64
0        1
38405    1
38406    4
38407    1
38408    1
        ..
34861    

In [18]:
dict = {'policy_tenure': change_df[0], 'age_of_car': change_df[1],'age_of_policyholder': change_df[2],'population_density': change_df[3],'make': change_df[4],'airbags': change_df[5],'displacement': change_df[6],'cylinder': change_df[7],'gear_box': change_df[8],'turning_radius': change_df[9],'length': change_df[10],'width': change_df[11],'height': change_df[12],'gross_weight': change_df[13],'ncap_rating': change_df[14]}
pd_change_df=pd.DataFrame(dict)
pd_change_df

,policy_tenure,age_of_car,age_of_policyholder,population_density,make,airbags,displacement,cylinder,gear_box,turning_radius,length,width,height,gross_weight,ncap_rating
0,2,2,4,1,3,3,2,2,4,2,2,2,2,2,2
1,1,2,1,1,3,3,2,2,4,2,2,2,2,2,2
2,3,2,2,3,4,3,4,4,4,4,3,4,2,3,4
3,3,1,1,1,3,3,2,2,4,2,2,2,2,2,2
4,2,2,1,1,3,3,2,2,4,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58587,2,3,1,4,4,3,4,4,4,4,3,4,2,3,4
58588,2,3,1,3,3,3,3,4,4,3,4,3,2,3,3
58589,3,2,4,4,3,3,2,2,4,2,2,2,2,2,2
58590,2,3,2,4,3,3,3,4,4,3,3,3,3,3,3


In [19]:
pd_change_df2=df.drop(['policy_id','is_claim','policy_tenure','age_of_car','age_of_policyholder','population_density','make','airbags','displacement','cylinder','gear_box','turning_radius','length','width','height','gross_weight','ncap_rating'],axis=1)
pd_change_df2

,area_cluster,segment,model,fuel_type,max_torque,max_power,engine_type,is_esc,is_adjustable_steering,is_tpms,...,is_rear_window_washer,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert
0,C1,A,M1,CNG,60Nm@3500rpm,40.36bhp@6000rpm,F8D Petrol Engine,No,No,No,...,No,No,No,No,No,Yes,No,No,No,Yes
38405,C3,A,M1,CNG,60Nm@3500rpm,40.36bhp@6000rpm,F8D Petrol Engine,No,No,No,...,No,No,No,No,No,Yes,No,No,No,Yes
38406,C8,B2,M5,Diesel,200Nm@3000rpm,88.77bhp@4000rpm,1.5 Turbocharged Revotorq,No,Yes,No,...,No,No,No,Yes,Yes,Yes,No,No,Yes,Yes
38407,C3,A,M1,CNG,60Nm@3500rpm,40.36bhp@6000rpm,F8D Petrol Engine,No,No,No,...,No,No,No,No,No,Yes,No,No,No,Yes
38408,C3,A,M1,CNG,60Nm@3500rpm,40.36bhp@6000rpm,F8D Petrol Engine,No,No,No,...,No,No,No,No,No,Yes,No,No,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34861,C17,B2,M5,Diesel,200Nm@3000rpm,88.77bhp@4000rpm,1.5 Turbocharged Revotorq,No,Yes,No,...,No,No,No,Yes,Yes,Yes,No,No,Yes,Yes
51164,C8,C1,M2,Petrol,113Nm@4400rpm,88.50bhp@6000rpm,1.2 L K12N Dualjet,Yes,Yes,No,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
34866,C2,A,M1,CNG,60Nm@3500rpm,40.36bhp@6000rpm,F8D Petrol Engine,No,No,No,...,No,No,No,No,No,Yes,No,No,No,Yes
6955,C10,B2,M6,Petrol,113Nm@4400rpm,88.50bhp@6000rpm,K Series Dual jet,No,Yes,No,...,No,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes


In [20]:
rf_df=pd.concat([pd_change_df,pd_change_df2],axis=1)
rf_df

,policy_tenure,age_of_car,age_of_policyholder,population_density,make,airbags,displacement,cylinder,gear_box,turning_radius,...,is_rear_window_washer,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert
0,2,2,4,1,3,3,2,2,4,2,...,No,No,No,No,No,Yes,No,No,No,Yes
1,1,2,1,1,3,3,2,2,4,2,...,No,No,No,No,No,Yes,No,No,No,Yes
2,3,2,2,3,4,3,4,4,4,4,...,No,No,No,No,No,Yes,No,No,No,Yes
3,3,1,1,1,3,3,2,2,4,2,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
4,2,2,1,1,3,3,2,2,4,2,...,No,No,No,Yes,Yes,Yes,No,Yes,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58587,2,3,1,4,4,3,4,4,4,4,...,No,No,No,Yes,Yes,Yes,No,Yes,Yes,Yes
58588,2,3,1,3,3,3,3,4,4,3,...,No,No,No,No,No,Yes,No,No,No,Yes
58589,3,2,4,4,3,3,2,2,4,2,...,No,No,No,No,No,Yes,No,No,No,Yes
58590,2,3,2,4,3,3,3,4,4,3,...,No,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes


In [21]:
for i in rf_df.columns:
    rf_df[i]=rf_df[i].astype(str)
rf_df

,policy_tenure,age_of_car,age_of_policyholder,population_density,make,airbags,displacement,cylinder,gear_box,turning_radius,...,is_rear_window_washer,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert
0,2,2,4,1,3,3,2,2,4,2,...,No,No,No,No,No,Yes,No,No,No,Yes
1,1,2,1,1,3,3,2,2,4,2,...,No,No,No,No,No,Yes,No,No,No,Yes
2,3,2,2,3,4,3,4,4,4,4,...,No,No,No,No,No,Yes,No,No,No,Yes
3,3,1,1,1,3,3,2,2,4,2,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
4,2,2,1,1,3,3,2,2,4,2,...,No,No,No,Yes,Yes,Yes,No,Yes,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58587,2,3,1,4,4,3,4,4,4,4,...,No,No,No,Yes,Yes,Yes,No,Yes,Yes,Yes
58588,2,3,1,3,3,3,3,4,4,3,...,No,No,No,No,No,Yes,No,No,No,Yes
58589,3,2,4,4,3,3,2,2,4,2,...,No,No,No,No,No,Yes,No,No,No,Yes
58590,2,3,2,4,3,3,3,4,4,3,...,No,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes


In [22]:
def as_categorical(df):
    newdf = df.copy()
    appdf = pd.DataFrame()
    for i in df.columns:
        if not isinstance(newdf[i][0],str):
            newls = newdf[i]<df[i].median()
            newdf[i] = newls.astype(int)
    return(newdf)

In [23]:
rf_df['is_claim']=df['is_claim']
df_0=rf_df[rf_df['is_claim']==0]
df_1=rf_df[rf_df['is_claim']==1]
df_0=shuffle(df_0)
df_1=shuffle(df_1)

In [24]:
test_df=pd.concat([df_0[:10969],df_1[:750]])
train_df=pd.concat([df_0[10969:],df_1[750:]])
train_feature=train_df.drop(['is_claim'],axis=1)
train_label=train_df['is_claim'][:]
test_feature=test_df.drop(['is_claim'],axis=1)
train_feature = as_categorical(train_feature)[:]

In [25]:
from math import log
def entropy(y):
        label_list,label_counts=np.unique(y,return_counts=True)
        cal_entropy=0
        for i in range(len(label_list)):
            cal_entropy=cal_entropy-(label_counts[i]/sum(label_counts))*log((label_counts[i]/sum(label_counts)),2)
        return cal_entropy
def info_gain(x,y):
        x_list,x_counts=np.unique(x,return_counts=True)
        info=entropy(y)*1
        for i in range(len(x_list)):
            info=info-(((x_counts[i]/sum(x_counts)))*entropy(y[x==x_list[i]]))
        return info
class node:
    def __init__(self,root,label=None,feature_name=None,feature=None):
        self.root=root
        self.label=label
        self.feature_name=feature_name
        self.tree={}
        self.outcome={
            'label:': self.label,
            'feature': self.feature_name,
            'tree': self.tree
        }
    def __repr__(self):
        return '{}'.format(self.result)

    def add_node(self, val, node):
        self.tree[val] = node

    def predict(self, df):
        if self.root is True:
            return self.label
        return self.tree[df[self.feature_name]].predict(df)
class decision_tree:
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon
        self._tree = {}
    def entropy(self,y):
        label_list,label_counts=np.unique(y,return_counts=True)
        cal_entropy=0
        for i in range(len(label_list)):
            cal_entropy=cal_entropy-(label_counts[i]/sum(label_counts))*log((label_counts[i]/sum(label_counts)),2)
        return cal_entropy
    
    def info_gain(self,x,y):
        x_list,x_counts=np.unique(x,return_counts=True)
        info=self.entropy(y)*1
        for i in range(len(x_list)):
            info=info-(((x_counts[i]/sum(x_counts)))*self.entropy(y[x==x_list[i]]))
        return info
    
    def max_info(self,df,y):
        list_name=[]
        temp_info_gain=[]
        for i in df.columns:
            list_name.append(i)
            temp_info_gain.append(self.info_gain(df[i],y))
        df_info=pd.DataFrame({"feature":list_name,"info_gain":temp_info_gain})
        return (df_info['feature'][df_info['info_gain'].idxmax()])
    
    def train_tree(self,df,y):
        #all labels are same
        if len(np.unique(y))==1:
            return node(root=True,label=y.iloc[0])
        #at least one label is not same as others
        value,count=np.unique(y,return_counts=True)
        temp_table=pd.DataFrame({"value":value,"count":count})
        #if all features are used-> root
        if len(df.columns)==0:
            return node(root=True,label=temp_table["value"][temp_table["count"].idxmax()])
        #exist feature to used
        max_info_feature=self.max_info(df,y)
        info_gain_feature=info_gain(df[max_info_feature],y)
        if info_gain_feature <= self.epsilon:
            return node(root=True,label=temp_table["value"][temp_table["count"].idxmax()])
        leave_feature=np.unique(df[max_info_feature])
        for i in leave_feature:
            subdf=df.loc[df[max_info_feature]==i].drop([max_info_feature],axis=1)
            suby=y[df[max_info_feature]==i]
            subtree=self.train_tree(subdf,suby)
            node_tree.add_node(i,subtree)
        return node_tree
    
    def fit(self,df,y):
        self._tree=self.train_tree(df,y)
        return self._tree
    
    def pred(self,df):
        predict=[]
        for i in range(len(df)):
            predict.append(self._tree.predict(df.iloc[i]))
        return predict

In [26]:
def RandomForest(train_df,train_label,test_df,ntree=100,split=0.5,epsilon=0.1):
    predict=[]
    for i in range(ntree):
        subdf=train_df.sample(frac=split,replace=False)
        sublabel=train_label[subdf.index.values]
        temp_tree=decision_tree(epsilon=epsilon)
        temp_tree.fit(subdf,sublabel)
        temp_tree_predict=temp_tree.pred(test_df)
        predict.append(temp_tree_predict)
    predict=pd.DataFrame(predict)
    result=[]
    for i in predict.columns:
        if(predict[i].mean()>=0.5):
            result.append(1)
        else:
            result.append(0)
    return result

In [27]:
temp_test_pred=RandomForest(train_feature,train_label,test_feature)

In [28]:
confusion_matrix(temp_test_pred,test_label)

array([[10969,   750],
       [    0,     0]], dtype=int64)

## sklearn RandomForest

In [29]:
test_df=pd.concat([df_0[:10969],df_1[:750]])
train_df=pd.concat([df_0[10969:],df_1[750:]])

In [30]:
def one_hot_encoding(df):
    appenddf = pd.DataFrame()
    temp = []
    for i in df.columns:
        temp.append(not isinstance(df[i][0],str))
        if isinstance(df[i][0],str):
            colclass = np.unique(df[i])
            for j in colclass:
                newname = i+"_"+j
                newlist = (df[i]==j).astype(int)
                appenddf[newname] = newlist
    new_df = pd.concat([df[df.columns[temp]],appenddf],axis=1)

    
    return(new_df)

In [31]:
onehot_df=one_hot_encoding(df.drop(['policy_id'],axis=1))

random.seed(6111058)
df_0=onehot_df[onehot_df['is_claim']==0]
df_1=onehot_df[onehot_df['is_claim']==1]
df_0=shuffle(df_0)
df_1=shuffle(df_1)

test_df=pd.concat([df_0[:10969],df_1[:750]])
train_df=pd.concat([df_0[10969:],df_1[750:]])
train_feature=train_df.drop(['is_claim'],axis=1)
train_label=train_df['is_claim'][:]
test_feature=test_df.drop(['is_claim'],axis=1)

C:\Users\andre\AppData\Local\Temp\ipykernel_12488\656572145.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  appenddf[newname] = newlist
C:\Users\andre\AppData\Local\Temp\ipykernel_12488\656572145.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  appenddf[newname] = newlist
C:\Users\andre\AppData\Local\Temp\ipykernel_12488\656572145.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once u

In [32]:
from sklearn import ensemble, metrics
forest = ensemble.RandomForestClassifier(n_estimators = 100)
forest_fit = forest.fit(train_feature, train_label)
pred_skforest=forest.predict(test_feature)
confusion_matrix(pred_skforest,test_label)

array([[10899,   744],
       [   70,     6]], dtype=int64)

## XGBoost

In [33]:
from xgboost import XGBClassifier
xgboost = XGBClassifier(n_estimators=100, learning_rate= 0.3)
xgboost.fit(train_feature, train_label)
predictxgb = xgboost.predict(test_feature)
confusion_matrix(predictxgb,test_label)

array([[10966,   750],
       [    3,     0]], dtype=int64)

## CatBoost

In [34]:
from catboost import CatBoostRegressor
catb = CatBoostRegressor(random_state=42,
                         loss_function='RMSE',
                         eval_metric='RMSE',
                         use_best_model=True)
catb.fit(train_feature, train_label,eval_set=(test_feature,test_label))
predictcatb = catb.predict(test_feature)

Learning rate set to 0.09316
0:	learn: 0.2442944	test: 0.2444105	best: 0.2444105 (0)	total: 149ms	remaining: 2m 28s
1:	learn: 0.2439806	test: 0.2441520	best: 0.2441520 (1)	total: 154ms	remaining: 1m 16s
2:	learn: 0.2437501	test: 0.2439563	best: 0.2439563 (2)	total: 158ms	remaining: 52.5s
3:	learn: 0.2435403	test: 0.2437875	best: 0.2437875 (3)	total: 163ms	remaining: 40.6s
4:	learn: 0.2433569	test: 0.2436475	best: 0.2436475 (4)	total: 167ms	remaining: 33.3s
5:	learn: 0.2431883	test: 0.2435226	best: 0.2435226 (5)	total: 172ms	remaining: 28.5s
6:	learn: 0.2430340	test: 0.2434450	best: 0.2434450 (6)	total: 177ms	remaining: 25.1s
7:	learn: 0.2429280	test: 0.2433591	best: 0.2433591 (7)	total: 182ms	remaining: 22.5s
8:	learn: 0.2428236	test: 0.2432829	best: 0.2432829 (8)	total: 187ms	remaining: 20.5s
9:	learn: 0.2427165	test: 0.2431883	best: 0.2431883 (9)	total: 191ms	remaining: 19s
10:	learn: 0.2426364	test: 0.2431462	best: 0.2431462 (10)	total: 197ms	remaining: 17.7s
11:	learn: 0.2425853	te

114:	learn: 0.2404825	test: 0.2427645	best: 0.2426750 (54)	total: 730ms	remaining: 5.62s
115:	learn: 0.2404662	test: 0.2427509	best: 0.2426750 (54)	total: 735ms	remaining: 5.6s
116:	learn: 0.2404246	test: 0.2427540	best: 0.2426750 (54)	total: 739ms	remaining: 5.58s
117:	learn: 0.2403993	test: 0.2427485	best: 0.2426750 (54)	total: 745ms	remaining: 5.57s
118:	learn: 0.2403875	test: 0.2427480	best: 0.2426750 (54)	total: 750ms	remaining: 5.55s
119:	learn: 0.2403781	test: 0.2427513	best: 0.2426750 (54)	total: 755ms	remaining: 5.53s
120:	learn: 0.2403652	test: 0.2427530	best: 0.2426750 (54)	total: 759ms	remaining: 5.52s
121:	learn: 0.2403478	test: 0.2427558	best: 0.2426750 (54)	total: 765ms	remaining: 5.5s
122:	learn: 0.2403176	test: 0.2427576	best: 0.2426750 (54)	total: 770ms	remaining: 5.49s
123:	learn: 0.2402958	test: 0.2427576	best: 0.2426750 (54)	total: 775ms	remaining: 5.47s
124:	learn: 0.2402757	test: 0.2427482	best: 0.2426750 (54)	total: 780ms	remaining: 5.46s
125:	learn: 0.2402638	t

227:	learn: 0.2383615	test: 0.2430699	best: 0.2426750 (54)	total: 1.28s	remaining: 4.34s
228:	learn: 0.2383462	test: 0.2430711	best: 0.2426750 (54)	total: 1.28s	remaining: 4.33s
229:	learn: 0.2383379	test: 0.2430752	best: 0.2426750 (54)	total: 1.29s	remaining: 4.32s
230:	learn: 0.2383205	test: 0.2430775	best: 0.2426750 (54)	total: 1.29s	remaining: 4.31s
231:	learn: 0.2383016	test: 0.2430822	best: 0.2426750 (54)	total: 1.3s	remaining: 4.3s
232:	learn: 0.2382820	test: 0.2430767	best: 0.2426750 (54)	total: 1.3s	remaining: 4.29s
233:	learn: 0.2382728	test: 0.2430790	best: 0.2426750 (54)	total: 1.31s	remaining: 4.29s
234:	learn: 0.2382407	test: 0.2430982	best: 0.2426750 (54)	total: 1.31s	remaining: 4.28s
235:	learn: 0.2382267	test: 0.2430984	best: 0.2426750 (54)	total: 1.32s	remaining: 4.27s
236:	learn: 0.2382141	test: 0.2430971	best: 0.2426750 (54)	total: 1.32s	remaining: 4.26s
237:	learn: 0.2381943	test: 0.2431024	best: 0.2426750 (54)	total: 1.33s	remaining: 4.25s
238:	learn: 0.2381827	te

337:	learn: 0.2365877	test: 0.2433074	best: 0.2426750 (54)	total: 1.83s	remaining: 3.58s
338:	learn: 0.2365752	test: 0.2432999	best: 0.2426750 (54)	total: 1.83s	remaining: 3.58s
339:	learn: 0.2365556	test: 0.2433082	best: 0.2426750 (54)	total: 1.84s	remaining: 3.57s
340:	learn: 0.2365338	test: 0.2433190	best: 0.2426750 (54)	total: 1.84s	remaining: 3.56s
341:	learn: 0.2365278	test: 0.2433166	best: 0.2426750 (54)	total: 1.85s	remaining: 3.56s
342:	learn: 0.2365181	test: 0.2433189	best: 0.2426750 (54)	total: 1.85s	remaining: 3.55s
343:	learn: 0.2364996	test: 0.2433269	best: 0.2426750 (54)	total: 1.86s	remaining: 3.54s
344:	learn: 0.2364968	test: 0.2433276	best: 0.2426750 (54)	total: 1.86s	remaining: 3.54s
345:	learn: 0.2364895	test: 0.2433323	best: 0.2426750 (54)	total: 1.87s	remaining: 3.53s
346:	learn: 0.2364829	test: 0.2433294	best: 0.2426750 (54)	total: 1.87s	remaining: 3.52s
347:	learn: 0.2364800	test: 0.2433295	best: 0.2426750 (54)	total: 1.88s	remaining: 3.52s
348:	learn: 0.2364686

451:	learn: 0.2350771	test: 0.2436255	best: 0.2426750 (54)	total: 2.38s	remaining: 2.89s
452:	learn: 0.2350656	test: 0.2436269	best: 0.2426750 (54)	total: 2.39s	remaining: 2.88s
453:	learn: 0.2350567	test: 0.2436298	best: 0.2426750 (54)	total: 2.39s	remaining: 2.88s
454:	learn: 0.2350558	test: 0.2436310	best: 0.2426750 (54)	total: 2.4s	remaining: 2.87s
455:	learn: 0.2350474	test: 0.2436387	best: 0.2426750 (54)	total: 2.4s	remaining: 2.87s
456:	learn: 0.2350354	test: 0.2436465	best: 0.2426750 (54)	total: 2.41s	remaining: 2.86s
457:	learn: 0.2350314	test: 0.2436524	best: 0.2426750 (54)	total: 2.41s	remaining: 2.85s
458:	learn: 0.2350228	test: 0.2436456	best: 0.2426750 (54)	total: 2.42s	remaining: 2.85s
459:	learn: 0.2349957	test: 0.2436450	best: 0.2426750 (54)	total: 2.42s	remaining: 2.84s
460:	learn: 0.2349873	test: 0.2436493	best: 0.2426750 (54)	total: 2.43s	remaining: 2.84s
461:	learn: 0.2349703	test: 0.2436517	best: 0.2426750 (54)	total: 2.43s	remaining: 2.83s
462:	learn: 0.2349646	t

559:	learn: 0.2336771	test: 0.2438940	best: 0.2426750 (54)	total: 2.93s	remaining: 2.3s
560:	learn: 0.2336735	test: 0.2438957	best: 0.2426750 (54)	total: 2.94s	remaining: 2.3s
561:	learn: 0.2336655	test: 0.2438968	best: 0.2426750 (54)	total: 2.94s	remaining: 2.29s
562:	learn: 0.2336484	test: 0.2439005	best: 0.2426750 (54)	total: 2.95s	remaining: 2.29s
563:	learn: 0.2336333	test: 0.2438987	best: 0.2426750 (54)	total: 2.95s	remaining: 2.28s
564:	learn: 0.2336157	test: 0.2439031	best: 0.2426750 (54)	total: 2.96s	remaining: 2.27s
565:	learn: 0.2336024	test: 0.2439082	best: 0.2426750 (54)	total: 2.96s	remaining: 2.27s
566:	learn: 0.2335784	test: 0.2439175	best: 0.2426750 (54)	total: 2.96s	remaining: 2.26s
567:	learn: 0.2335635	test: 0.2439222	best: 0.2426750 (54)	total: 2.97s	remaining: 2.26s
568:	learn: 0.2335481	test: 0.2439240	best: 0.2426750 (54)	total: 2.98s	remaining: 2.25s
569:	learn: 0.2335408	test: 0.2439276	best: 0.2426750 (54)	total: 2.98s	remaining: 2.25s
570:	learn: 0.2335309	t

668:	learn: 0.2323973	test: 0.2441674	best: 0.2426750 (54)	total: 3.48s	remaining: 1.72s
669:	learn: 0.2323909	test: 0.2441647	best: 0.2426750 (54)	total: 3.48s	remaining: 1.72s
670:	learn: 0.2323840	test: 0.2441624	best: 0.2426750 (54)	total: 3.49s	remaining: 1.71s
671:	learn: 0.2323705	test: 0.2441630	best: 0.2426750 (54)	total: 3.5s	remaining: 1.71s
672:	learn: 0.2323496	test: 0.2441696	best: 0.2426750 (54)	total: 3.5s	remaining: 1.7s
673:	learn: 0.2323236	test: 0.2441733	best: 0.2426750 (54)	total: 3.5s	remaining: 1.7s
674:	learn: 0.2323030	test: 0.2441764	best: 0.2426750 (54)	total: 3.51s	remaining: 1.69s
675:	learn: 0.2322740	test: 0.2441739	best: 0.2426750 (54)	total: 3.52s	remaining: 1.69s
676:	learn: 0.2322543	test: 0.2441823	best: 0.2426750 (54)	total: 3.52s	remaining: 1.68s
677:	learn: 0.2322433	test: 0.2441875	best: 0.2426750 (54)	total: 3.53s	remaining: 1.67s
678:	learn: 0.2322323	test: 0.2441929	best: 0.2426750 (54)	total: 3.53s	remaining: 1.67s
679:	learn: 0.2322190	test

779:	learn: 0.2309578	test: 0.2444626	best: 0.2426750 (54)	total: 4.03s	remaining: 1.14s
780:	learn: 0.2309372	test: 0.2444590	best: 0.2426750 (54)	total: 4.04s	remaining: 1.13s
781:	learn: 0.2309332	test: 0.2444597	best: 0.2426750 (54)	total: 4.04s	remaining: 1.13s
782:	learn: 0.2309159	test: 0.2444606	best: 0.2426750 (54)	total: 4.05s	remaining: 1.12s
783:	learn: 0.2308978	test: 0.2444667	best: 0.2426750 (54)	total: 4.05s	remaining: 1.12s
784:	learn: 0.2308888	test: 0.2444679	best: 0.2426750 (54)	total: 4.06s	remaining: 1.11s
785:	learn: 0.2308854	test: 0.2444715	best: 0.2426750 (54)	total: 4.06s	remaining: 1.1s
786:	learn: 0.2308680	test: 0.2444772	best: 0.2426750 (54)	total: 4.07s	remaining: 1.1s
787:	learn: 0.2308528	test: 0.2444782	best: 0.2426750 (54)	total: 4.07s	remaining: 1.09s
788:	learn: 0.2308431	test: 0.2444844	best: 0.2426750 (54)	total: 4.08s	remaining: 1.09s
789:	learn: 0.2308270	test: 0.2444879	best: 0.2426750 (54)	total: 4.08s	remaining: 1.08s
790:	learn: 0.2308094	t

888:	learn: 0.2297179	test: 0.2447474	best: 0.2426750 (54)	total: 4.58s	remaining: 572ms
889:	learn: 0.2297099	test: 0.2447513	best: 0.2426750 (54)	total: 4.58s	remaining: 567ms
890:	learn: 0.2296937	test: 0.2447581	best: 0.2426750 (54)	total: 4.59s	remaining: 562ms
891:	learn: 0.2296906	test: 0.2447585	best: 0.2426750 (54)	total: 4.59s	remaining: 556ms
892:	learn: 0.2296688	test: 0.2447623	best: 0.2426750 (54)	total: 4.6s	remaining: 551ms
893:	learn: 0.2296647	test: 0.2447662	best: 0.2426750 (54)	total: 4.6s	remaining: 546ms
894:	learn: 0.2296617	test: 0.2447680	best: 0.2426750 (54)	total: 4.61s	remaining: 541ms
895:	learn: 0.2296547	test: 0.2447721	best: 0.2426750 (54)	total: 4.61s	remaining: 536ms
896:	learn: 0.2296421	test: 0.2447725	best: 0.2426750 (54)	total: 4.62s	remaining: 530ms
897:	learn: 0.2296421	test: 0.2447725	best: 0.2426750 (54)	total: 4.62s	remaining: 525ms
898:	learn: 0.2296280	test: 0.2447767	best: 0.2426750 (54)	total: 4.63s	remaining: 520ms
899:	learn: 0.2296190	t

999:	learn: 0.2286123	test: 0.2450519	best: 0.2426750 (54)	total: 5.13s	remaining: 0us

bestTest = 0.2426750053
bestIteration = 54

Shrink model to first 55 iterations.


In [35]:
cat_pred=[]
for i in range(len(predictcatb)):
    if(predictcatb[i]>=0.5):
        cat_pred.append(1)
    else:
        cat_pred.append(0)
confusion_matrix(cat_pred,test_label)

array([[10969,   750],
       [    0,     0]], dtype=int64)

In [37]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
classifier = lgb.LGBMClassifier(objective = 'binary', 
                                learning_rate = 0.05, 
                                n_estimators = 100, 
                                random_state=0)
classifier.fit(train_feature, train_label)
lightgbm_pred = classifier.predict(test_feature)
confusion_matrix(lightgbm_pred.tolist(), test_label)

array([[10969,   750],
       [    0,     0]], dtype=int64)

## CV

In [116]:
def RF_CV_process(df,k):
    temp=list(range(len(df)))
    random.shuffle(temp)
    list_temp=[]
    for i in range(k):
        list_temp.append(temp[i*round(len(temp)/k):(i+1)*round(len(temp)/k)])
    pred_acc=[]
    for i in range(k):
        test_df=df.iloc[list_temp[i],:]
        train_df=pd.concat([df,test_df]).drop_duplicates(keep=False)
        test_feature=test_df.drop(['is_claim'],axis=1)
        test_label=test_df['is_claim']
        train_feature=train_df.drop(['is_claim'],axis=1)
        train_label=train_df['is_claim']
        cv_pred=RandomForest(train_feature,train_label,test_feature)
        tn, fp, fn, tp = confusion_matrix(cv_pred,test_label).ravel()
        acc=(tn+tp)/(tn+tp+fn+fp)
        pred_acc.append(acc)
    return pred_acc

In [120]:
CV_RF_acc3=RF_CV_process(rf_df,3)

In [121]:
CV_RF_acc5=RF_CV_process(rf_df,5)

In [122]:
CV_RF_acc10=RF_CV_process(rf_df,10)

In [161]:
from statistics import mean
CV_RF_acc3

[0.9348215657160411, 0.9341047565408838, 0.9391705069124424]

In [162]:
mean(CV_RF_acc3)

0.9360322763897891

In [163]:
mean(CV_RF_acc5)

0.9360300392558457

In [164]:
mean(CV_RF_acc10)

0.9360300392558457

In [130]:
def NB_CV_process(df,k):
    temp=list(range(len(df)))
    random.shuffle(temp)
    list_temp=[]
    for i in range(k):
        list_temp.append(temp[i*round(len(temp)/k):(i+1)*round(len(temp)/k)])
    pred_acc=[]
    for i in range(k):
        test_df=df.iloc[list_temp[i],:]
        train_df=pd.concat([df,test_df]).drop_duplicates(keep=False)
        test_feature=test_df.drop(['is_claim'],axis=1)
        test_label=test_df['is_claim']
        train_feature=train_df.drop(['is_claim'],axis=1)
        train_label=train_df['is_claim']
        cv_pred=naive_bayes(train_feature,train_label,test_feature)
        tn, fp, fn, tp = confusion_matrix(cv_pred,test_label).ravel()
        acc=(tn+tp)/(tn+tp+fn+fp)
        pred_acc.append(acc)
    return pred_acc

In [131]:
def SK_RF_CV(df,k):
    temp=list(range(len(df)))
    random.shuffle(temp)
    list_temp=[]
    for i in range(k):
        list_temp.append(temp[i*round(len(temp)/k):(i+1)*round(len(temp)/k)])
    pred_acc=[]
    for i in range(k):
        test_df=df.iloc[list_temp[i],:]
        train_df=pd.concat([df,test_df]).drop_duplicates(keep=False)
        test_feature=test_df.drop(['is_claim'],axis=1)
        test_label=test_df['is_claim']
        train_feature=train_df.drop(['is_claim'],axis=1)
        train_label=train_df['is_claim']
        forest = ensemble.RandomForestClassifier(n_estimators = 100)
        forest_fit = forest.fit(train_feature, train_label)
        pred_skforest=forest.predict(test_feature)
        tn, fp, fn, tp = confusion_matrix(pred_skforest,test_label).ravel()
        acc=(tn+tp)/(tn+tp+fn+fp)
        pred_acc.append(acc)
    return pred_acc

In [132]:
cv_onehot3=SK_RF_CV(onehot_df,3)
cv_onehot5=SK_RF_CV(onehot_df,5)
cv_onehot10=SK_RF_CV(onehot_df,10)

19531
19531
19530
11718
11718
11718
11718
11718
5859
5859
5859
5859
5859
5859
5859
5859
5859
5859


In [165]:
mean(cv_onehot3)

0.9313387475295509

In [166]:
mean(cv_onehot5)

0.9310121181088923

In [167]:
mean(cv_onehot10)

0.9309609148318826

In [136]:
def XGB_CV(df,k):
    temp=list(range(len(df)))
    random.shuffle(temp)
    list_temp=[]
    for i in range(k):
        list_temp.append(temp[i*round(len(temp)/k):(i+1)*round(len(temp)/k)])
    pred_acc=[]
    for i in range(k):
        test_df=df.iloc[list_temp[i],:]
        train_df=pd.concat([df,test_df]).drop_duplicates(keep=False)
        test_feature=test_df.drop(['is_claim'],axis=1)
        test_label=test_df['is_claim']
        train_feature=train_df.drop(['is_claim'],axis=1)
        train_label=train_df['is_claim']
        xgboost.fit(train_feature, train_label)
        predictxgb = xgboost.predict(test_feature)
        tn, fp, fn, tp = confusion_matrix(predictxgb,test_label).ravel()
        acc=(tn+tp)/(tn+tp+fn+fp)
        pred_acc.append(acc)
    return pred_acc

In [139]:
cv_XGB3=XGB_CV(onehot_df,3)
cv_XGB5=XGB_CV(onehot_df,5)
cv_XGB10=XGB_CV(onehot_df,10)

In [168]:
mean(cv_XGB3)

0.9357761935898264

In [169]:
mean(cv_XGB5)

0.9358422939068101

In [170]:
mean(cv_XGB10)

0.9358934971838198

In [137]:
def CATB_CV(df,k):
    temp=list(range(len(df)))
    random.shuffle(temp)
    list_temp=[]
    for i in range(k):
        list_temp.append(temp[i*round(len(temp)/k):(i+1)*round(len(temp)/k)])
    pred_acc=[]
    for i in range(k):
        test_df=df.iloc[list_temp[i],:]
        train_df=pd.concat([df,test_df]).drop_duplicates(keep=False)
        test_feature=test_df.drop(['is_claim'],axis=1)
        test_label=test_df['is_claim']
        train_feature=train_df.drop(['is_claim'],axis=1)
        train_label=train_df['is_claim']
        catb = CatBoostRegressor(random_state=42,
                         loss_function='RMSE',
                         eval_metric='RMSE',
                         use_best_model=True)
        catb.fit(train_feature, train_label,eval_set=(test_feature,test_label))
        predictcatb = catb.predict(test_feature)
        cat_pred=[]
        for i in range(len(predictcatb)):
            if(predictcatb[i]>=0.5):
                cat_pred.append(1)
            else:
                cat_pred.append(0)
        tn, fp, fn, tp = confusion_matrix(cat_pred,test_label).ravel()
        acc=(tn+tp)/(tn+tp+fn+fp)
        pred_acc.append(acc)
    return pred_acc

In [143]:
cv_CATB3=CATB_CV(onehot_df,3)
cv_CATB5=CATB_CV(onehot_df,5)
cv_CATB10=CATB_CV(onehot_df,10)

Learning rate set to 0.090531
0:	learn: 0.2406404	test: 0.2516336	best: 0.2516336 (0)	total: 4.28ms	remaining: 4.28s
1:	learn: 0.2403569	test: 0.2513482	best: 0.2513482 (1)	total: 8.51ms	remaining: 4.25s
2:	learn: 0.2401493	test: 0.2511327	best: 0.2511327 (2)	total: 12.3ms	remaining: 4.1s
3:	learn: 0.2399571	test: 0.2509790	best: 0.2509790 (3)	total: 16ms	remaining: 3.97s
4:	learn: 0.2397891	test: 0.2508099	best: 0.2508099 (4)	total: 19.8ms	remaining: 3.95s
5:	learn: 0.2396389	test: 0.2506452	best: 0.2506452 (5)	total: 24.1ms	remaining: 3.99s
6:	learn: 0.2394971	test: 0.2504975	best: 0.2504975 (6)	total: 28ms	remaining: 3.97s
7:	learn: 0.2394019	test: 0.2503934	best: 0.2503934 (7)	total: 32.2ms	remaining: 4s
8:	learn: 0.2393409	test: 0.2503740	best: 0.2503740 (8)	total: 37.1ms	remaining: 4.08s
9:	learn: 0.2392560	test: 0.2502804	best: 0.2502804 (9)	total: 41.3ms	remaining: 4.09s
10:	learn: 0.2391590	test: 0.2502052	best: 0.2502052 (10)	total: 45.7ms	remaining: 4.11s
11:	learn: 0.239102

130:	learn: 0.2365854	test: 0.2496294	best: 0.2495976 (98)	total: 541ms	remaining: 3.59s
131:	learn: 0.2365688	test: 0.2496266	best: 0.2495976 (98)	total: 546ms	remaining: 3.59s
132:	learn: 0.2365457	test: 0.2496306	best: 0.2495976 (98)	total: 550ms	remaining: 3.59s
133:	learn: 0.2365296	test: 0.2496375	best: 0.2495976 (98)	total: 555ms	remaining: 3.59s
134:	learn: 0.2365034	test: 0.2496431	best: 0.2495976 (98)	total: 559ms	remaining: 3.58s
135:	learn: 0.2364617	test: 0.2496358	best: 0.2495976 (98)	total: 564ms	remaining: 3.58s
136:	learn: 0.2364381	test: 0.2496319	best: 0.2495976 (98)	total: 568ms	remaining: 3.58s
137:	learn: 0.2364079	test: 0.2496329	best: 0.2495976 (98)	total: 572ms	remaining: 3.57s
138:	learn: 0.2363882	test: 0.2496361	best: 0.2495976 (98)	total: 576ms	remaining: 3.57s
139:	learn: 0.2363669	test: 0.2496363	best: 0.2495976 (98)	total: 581ms	remaining: 3.57s
140:	learn: 0.2363447	test: 0.2496385	best: 0.2495976 (98)	total: 585ms	remaining: 3.56s
141:	learn: 0.2363127

256:	learn: 0.2336733	test: 0.2499919	best: 0.2495976 (98)	total: 1.09s	remaining: 3.15s
257:	learn: 0.2336583	test: 0.2499863	best: 0.2495976 (98)	total: 1.09s	remaining: 3.14s
258:	learn: 0.2336361	test: 0.2499893	best: 0.2495976 (98)	total: 1.1s	remaining: 3.14s
259:	learn: 0.2336231	test: 0.2499925	best: 0.2495976 (98)	total: 1.1s	remaining: 3.14s
260:	learn: 0.2335988	test: 0.2499936	best: 0.2495976 (98)	total: 1.11s	remaining: 3.13s
261:	learn: 0.2335801	test: 0.2499862	best: 0.2495976 (98)	total: 1.11s	remaining: 3.13s
262:	learn: 0.2335554	test: 0.2499926	best: 0.2495976 (98)	total: 1.12s	remaining: 3.13s
263:	learn: 0.2335311	test: 0.2499881	best: 0.2495976 (98)	total: 1.12s	remaining: 3.12s
264:	learn: 0.2335089	test: 0.2499953	best: 0.2495976 (98)	total: 1.13s	remaining: 3.12s
265:	learn: 0.2334931	test: 0.2500011	best: 0.2495976 (98)	total: 1.13s	remaining: 3.12s
266:	learn: 0.2334786	test: 0.2499993	best: 0.2495976 (98)	total: 1.13s	remaining: 3.11s
267:	learn: 0.2334609	t

381:	learn: 0.2314873	test: 0.2503708	best: 0.2495976 (98)	total: 1.64s	remaining: 2.65s
382:	learn: 0.2314781	test: 0.2503750	best: 0.2495976 (98)	total: 1.64s	remaining: 2.65s
383:	learn: 0.2314627	test: 0.2503809	best: 0.2495976 (98)	total: 1.65s	remaining: 2.64s
384:	learn: 0.2314398	test: 0.2503966	best: 0.2495976 (98)	total: 1.65s	remaining: 2.64s
385:	learn: 0.2314282	test: 0.2503981	best: 0.2495976 (98)	total: 1.66s	remaining: 2.63s
386:	learn: 0.2314078	test: 0.2503995	best: 0.2495976 (98)	total: 1.66s	remaining: 2.63s
387:	learn: 0.2313896	test: 0.2504025	best: 0.2495976 (98)	total: 1.66s	remaining: 2.63s
388:	learn: 0.2313724	test: 0.2504098	best: 0.2495976 (98)	total: 1.67s	remaining: 2.62s
389:	learn: 0.2313713	test: 0.2504098	best: 0.2495976 (98)	total: 1.67s	remaining: 2.62s
390:	learn: 0.2313522	test: 0.2504134	best: 0.2495976 (98)	total: 1.68s	remaining: 2.61s
391:	learn: 0.2313305	test: 0.2504171	best: 0.2495976 (98)	total: 1.68s	remaining: 2.61s
392:	learn: 0.2312986

508:	learn: 0.2294698	test: 0.2508015	best: 0.2495976 (98)	total: 2.19s	remaining: 2.11s
509:	learn: 0.2294651	test: 0.2508049	best: 0.2495976 (98)	total: 2.19s	remaining: 2.1s
510:	learn: 0.2294560	test: 0.2508081	best: 0.2495976 (98)	total: 2.2s	remaining: 2.1s
511:	learn: 0.2294496	test: 0.2508101	best: 0.2495976 (98)	total: 2.2s	remaining: 2.1s
512:	learn: 0.2294359	test: 0.2508154	best: 0.2495976 (98)	total: 2.21s	remaining: 2.09s
513:	learn: 0.2294106	test: 0.2508218	best: 0.2495976 (98)	total: 2.21s	remaining: 2.09s
514:	learn: 0.2293824	test: 0.2508298	best: 0.2495976 (98)	total: 2.21s	remaining: 2.08s
515:	learn: 0.2293582	test: 0.2508295	best: 0.2495976 (98)	total: 2.22s	remaining: 2.08s
516:	learn: 0.2293382	test: 0.2508405	best: 0.2495976 (98)	total: 2.22s	remaining: 2.08s
517:	learn: 0.2293227	test: 0.2508337	best: 0.2495976 (98)	total: 2.23s	remaining: 2.07s
518:	learn: 0.2293062	test: 0.2508355	best: 0.2495976 (98)	total: 2.23s	remaining: 2.07s
519:	learn: 0.2292879	test

...

461:	learn: 0.2359028	test: 0.2423051	best: 0.2408896 (71)	total: 2.38s	remaining: 2.77s
462:	learn: 0.2358932	test: 0.2423085	best: 0.2408896 (71)	total: 2.38s	remaining: 2.76s
463:	learn: 0.2358814	test: 0.2423101	best: 0.2408896 (71)	total: 2.39s	remaining: 2.76s
464:	learn: 0.2358676	test: 0.2423127	best: 0.2408896 (71)	total: 2.39s	remaining: 2.75s
465:	learn: 0.2358527	test: 0.2423134	best: 0.2408896 (71)	total: 2.4s	remaining: 2.75s
466:	learn: 0.2358446	test: 0.2423183	best: 0.2408896 (71)	total: 2.4s	remaining: 2.74s
467:	learn: 0.2358309	test: 0.2423234	best: 0.2408896 (71)	total: 2.41s	remaining: 2.74s
468:	learn: 0.2358234	test: 0.2423274	best: 0.2408896 (71)	total: 2.41s	remaining: 2.73s
469:	learn: 0.2358192	test: 0.2423284	best: 0.2408896 (71)	total: 2.42s	remaining: 2.73s
470:	learn: 0.2358076	test: 0.2423309	best: 0.2408896 (71)	total: 2.42s	remaining: 2.72s
471:	learn: 0.2357981	test: 0.2423310	best: 0.2408896 (71)	total: 2.43s	remaining: 2.72s
472:	learn: 0.2357782	t

571:	learn: 0.2345486	test: 0.2426801	best: 0.2408896 (71)	total: 2.93s	remaining: 2.19s
572:	learn: 0.2345256	test: 0.2426768	best: 0.2408896 (71)	total: 2.94s	remaining: 2.19s
573:	learn: 0.2345109	test: 0.2426829	best: 0.2408896 (71)	total: 2.94s	remaining: 2.18s
574:	learn: 0.2344997	test: 0.2426813	best: 0.2408896 (71)	total: 2.94s	remaining: 2.18s
575:	learn: 0.2344825	test: 0.2426906	best: 0.2408896 (71)	total: 2.95s	remaining: 2.17s
576:	learn: 0.2344732	test: 0.2426935	best: 0.2408896 (71)	total: 2.96s	remaining: 2.17s
577:	learn: 0.2344560	test: 0.2426905	best: 0.2408896 (71)	total: 2.96s	remaining: 2.16s
578:	learn: 0.2344367	test: 0.2427042	best: 0.2408896 (71)	total: 2.96s	remaining: 2.15s
579:	learn: 0.2344279	test: 0.2427112	best: 0.2408896 (71)	total: 2.97s	remaining: 2.15s
580:	learn: 0.2344174	test: 0.2427064	best: 0.2408896 (71)	total: 2.98s	remaining: 2.15s
581:	learn: 0.2344042	test: 0.2427076	best: 0.2408896 (71)	total: 2.98s	remaining: 2.14s
582:	learn: 0.2343817

678:	learn: 0.2333736	test: 0.2429495	best: 0.2408896 (71)	total: 3.48s	remaining: 1.64s
679:	learn: 0.2333628	test: 0.2429550	best: 0.2408896 (71)	total: 3.48s	remaining: 1.64s
680:	learn: 0.2333396	test: 0.2429449	best: 0.2408896 (71)	total: 3.49s	remaining: 1.63s
681:	learn: 0.2333364	test: 0.2429502	best: 0.2408896 (71)	total: 3.49s	remaining: 1.63s
682:	learn: 0.2333255	test: 0.2429679	best: 0.2408896 (71)	total: 3.5s	remaining: 1.62s
683:	learn: 0.2333199	test: 0.2429689	best: 0.2408896 (71)	total: 3.5s	remaining: 1.62s
684:	learn: 0.2333113	test: 0.2429694	best: 0.2408896 (71)	total: 3.51s	remaining: 1.61s
685:	learn: 0.2333032	test: 0.2429779	best: 0.2408896 (71)	total: 3.51s	remaining: 1.61s
686:	learn: 0.2332905	test: 0.2429923	best: 0.2408896 (71)	total: 3.52s	remaining: 1.6s
687:	learn: 0.2332767	test: 0.2430010	best: 0.2408896 (71)	total: 3.52s	remaining: 1.6s
688:	learn: 0.2332674	test: 0.2430064	best: 0.2408896 (71)	total: 3.53s	remaining: 1.59s
689:	learn: 0.2332554	tes

788:	learn: 0.2322529	test: 0.2432002	best: 0.2408896 (71)	total: 4.03s	remaining: 1.08s
789:	learn: 0.2322428	test: 0.2431933	best: 0.2408896 (71)	total: 4.04s	remaining: 1.07s
790:	learn: 0.2322365	test: 0.2432000	best: 0.2408896 (71)	total: 4.05s	remaining: 1.07s
791:	learn: 0.2322319	test: 0.2431992	best: 0.2408896 (71)	total: 4.05s	remaining: 1.06s
792:	learn: 0.2322223	test: 0.2431967	best: 0.2408896 (71)	total: 4.06s	remaining: 1.06s
793:	learn: 0.2322138	test: 0.2432002	best: 0.2408896 (71)	total: 4.06s	remaining: 1.05s
794:	learn: 0.2322048	test: 0.2432115	best: 0.2408896 (71)	total: 4.07s	remaining: 1.05s
795:	learn: 0.2321963	test: 0.2432034	best: 0.2408896 (71)	total: 4.07s	remaining: 1.04s
796:	learn: 0.2321929	test: 0.2432036	best: 0.2408896 (71)	total: 4.08s	remaining: 1.04s
797:	learn: 0.2321882	test: 0.2432011	best: 0.2408896 (71)	total: 4.08s	remaining: 1.03s
798:	learn: 0.2321811	test: 0.2432040	best: 0.2408896 (71)	total: 4.09s	remaining: 1.03s
799:	learn: 0.2321713

894:	learn: 0.2312336	test: 0.2434446	best: 0.2408896 (71)	total: 4.58s	remaining: 538ms
895:	learn: 0.2312270	test: 0.2434438	best: 0.2408896 (71)	total: 4.59s	remaining: 533ms
896:	learn: 0.2312189	test: 0.2434426	best: 0.2408896 (71)	total: 4.59s	remaining: 527ms
897:	learn: 0.2312149	test: 0.2434415	best: 0.2408896 (71)	total: 4.6s	remaining: 522ms
898:	learn: 0.2312066	test: 0.2434351	best: 0.2408896 (71)	total: 4.6s	remaining: 517ms
899:	learn: 0.2311976	test: 0.2434503	best: 0.2408896 (71)	total: 4.61s	remaining: 512ms
900:	learn: 0.2311905	test: 0.2434468	best: 0.2408896 (71)	total: 4.61s	remaining: 507ms
901:	learn: 0.2311776	test: 0.2434439	best: 0.2408896 (71)	total: 4.62s	remaining: 502ms
902:	learn: 0.2311743	test: 0.2434487	best: 0.2408896 (71)	total: 4.62s	remaining: 497ms
903:	learn: 0.2311601	test: 0.2434486	best: 0.2408896 (71)	total: 4.63s	remaining: 492ms
904:	learn: 0.2311552	test: 0.2434569	best: 0.2408896 (71)	total: 4.63s	remaining: 487ms
905:	learn: 0.2311503	t

In [171]:
mean(cv_CATB3)

0.936032237065169

In [172]:
mean(cv_CATB5)

0.9360300392558457

In [173]:
mean(cv_CATB10)

0.9360300392558457

In [138]:
def LGBM_CV(df,k):
    temp=list(range(len(df)))
    random.shuffle(temp)
    list_temp=[]
    for i in range(k):
        list_temp.append(temp[i*round(len(temp)/k):(i+1)*round(len(temp)/k)])
    pred_acc=[]
    for i in range(k):
        test_df=df.iloc[list_temp[i],:]
        train_df=pd.concat([df,test_df]).drop_duplicates(keep=False)
        test_feature=test_df.drop(['is_claim'],axis=1)
        test_label=test_df['is_claim']
        train_feature=train_df.drop(['is_claim'],axis=1)
        train_label=train_df['is_claim']
        classifier = lgb.LGBMClassifier(objective = 'binary', 
                                learning_rate = 0.05, 
                                n_estimators = 100, 
                                random_state=0)
        classifier.fit(train_feature, train_label)
        lightgbm_pred = classifier.predict(test_feature)
        tn, fp, fn, tp = confusion_matrix(lightgbm_pred,test_label).ravel()
        acc=(tn+tp)/(tn+tp+fn+fp)
        pred_acc.append(acc)
    return pred_acc

In [147]:
cv_LGBM3=LGBM_CV(onehot_df,3)
cv_LGBM5=LGBM_CV(onehot_df,5)
cv_LGBM10=LGBM_CV(onehot_df,10)

In [174]:
mean(cv_LGBM3)

0.9360322213353209

In [175]:
mean(cv_LGBM5)

0.9360300392558457

In [176]:
mean(cv_LGBM10)

0.9360300392558457

In [154]:
naive_df=df.drop(['policy_id'],axis=1)
cv_NB3=NB_CV_process(naive_df,3)
cv_NB5=NB_CV_process(naive_df,5)
cv_NB10=NB_CV_process(naive_df,10)

19531
19531
19530
11718
11718
11718
11718
11718
5859
5859
5859
5859
5859
5859
5859
5859
5859
5859


In [177]:
mean(cv_NB3)

0.93603218026294

In [178]:
mean(cv_NB5)

0.9360300392558457

In [179]:
mean(cv_NB10)

0.9360300392558457

In [180]:
len(df_0)

54844

In [181]:
len(df_1)

3748